In [7]:
import nltk
import json
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import re
import pandas as pd

## Los Datos

### Inicialmente se obtienen los datos y se filtran solo por tweets clasificados como negativos y positivos. Los tweets neutros fue la clasificación donde se incluyeron aquellos que no podian ser definidos como positivos o negativos o aquellos que no tenian carga emocional (por ejemplo, titulares sin tendencia) 

In [9]:
df = pd.read_excel('data/Tweets_clasificados_dividido2.xlsx')
df = df[((df['Categoria'] == 'Positivo') | (df['Categoria'] == 'Negativo'))]
df.head(2)

,Id,Tweet,Categoria1,Categoria,Columna1,Fecha,Usuario,Ironia
0,1807,- No politicemos la vacuna\n- Spot de La Matan...,Positivo,Positivo,NaN,2021-06-24 02:45:05,Leandro Landin,NO
1,1667,#23Jun #Internacionales El presidente de Argen...,Positivo,Positivo,NaN,2021-06-23 22:43:46,Primicias24,NO


## Train y Test

### División de los datos para entrenamiento y evaluacion. La separacion es de 80% de los datos para entrenamiento y 20% para testeo

In [10]:
df_train, df_test =train_test_split(df,test_size=0.2,random_state=123, stratify=df['Categoria'])

### Se crea el corpus de palabras de los tweets (tanto positivos como negativos) para su posterior uso para verificar presencia ante los nuevos Tweets y determinar su clasificación

In [27]:
corpus_positivo = ""
for text in df_train[df_train['Categoria'] == "Positivo"]["Tweet"]:
    corpus_positivo += text + ' '

corpus_positivo = corpus_positivo.lower()
corpus_positivo[:1000]

'en la provincia ya hay 5 millones de bonaerenses vacunadas y vacunados, y acá en alberti mañana llegaremos a las 6000 primeras dosis aplicadas. somos #5millonesdesonrisas. tenemos 5 millones de esperanzas. falta poco para volver a abrazarnos!!!gracias @alferdez @kicillofok!!! y muchisimas gracias presidente señor alberto fernández!!! @lanacion hay segunda dosis, de echo llegaron 500 mil y mañana arranca otro avión rumbo a rusia. más allá del viaje le agradezco al señor presidente alberto fernández por fabricarla....la nación es macri, miente política📦| junto a @alferdez , el diputado nacional por tucumán,  @cisneros_tuc , celebró el 45% de aumento salarial alcanzado por la bancaria, defendió la gestión del gobierno nacional y ratificó su incondicional apoyo al gobernador @juanmanzurok .\nhttps://t.co/oiolp1av0r #políticaexterior | victoria tolosa paz: claro que hay democracia en #venezuela\n\npara la funcionaria @vtolosapaz, alberto fernández fue coherente en su posicionamiento  https

In [28]:
corpus_negativo = ""
for text in df_train[df_train['Categoria'] == "Negativo"]["Tweet"]:
    corpus_negativo += text + ' '

corpus_negativo = corpus_negativo.lower()
corpus_negativo[:1000]

'me hacen reír los que le piden a un mafioso como alberto fernández, que no pierda los estribos... o que no grite... o que le haya cuidado el culo al hijo... no entienden que estamos hablando de un peroncho??? ¡así lo dijo! “no hay vacuna que cure la mente retrógrada del socialismo”: los firmes comentarios de bolsonaro sobre alberto fernández y nicolás maduro (+video) #23jun #venezuela  https://t.co/u6voa0qtq8 @marcelopuella urge definir el nosotros y el ellos. esta, marcelo, es la táctica del tero. el candidato de la oligarquía está en campaña desde hace meses con respaldo de mm, af y biden: sm https://t.co/6rzkjeuqqj https://t.co/mm4jp81mjj 706 muertes y a este ritmo vamos a promediar en 1 o 2 semanas las 1000 por día.\nmientras tanto alberto fernández permite entrada de viajeros vía ezeiza.\ncomo antes permitió que se hicieran manifestaciones contra la cuarentena, alentados por tn, a24, ln+.\nno quiero pensar mal pero #elvirussonellos los cipayos vendepatria de siempre, boicoteando 

### Se realiza la tokenización en palabras de los corpus

In [29]:
tokenizador_expreg = nltk.regexp_tokenize
palabras_positivo = tokenizador_expreg(corpus_positivo, pattern='\w+')
palabras_negativo = tokenizador_expreg(corpus_negativo, pattern='\w+') 

### Se realiza la eliminacion de las Stopwords procurando que las palabras que se utilicen para la valoración no sean palabras que carezcan de peso

In [30]:
def clear_stop_words(palabras):
    palabras_limpias = [x for x in palabras if (x not in stop_words) & (len(x) > 1 | x.isalnum())]
    return palabras_limpias

In [31]:
stop_words = stopwords.words('spanish')
palabras_positivo = clear_stop_words(palabras_positivo)
palabras_negativo = clear_stop_words(palabras_negativo)

### Un proceso de fuerte peso en este clasificador es eliminalas palabras que son comunes a ambos corpus. Con ello se pretende llevar a la mimina epresion los mismos para que solo queden las palabras mas significativas para la comparación

In [32]:
no_comunes_p = [x for x in palabras_positivo if (x not in (palabras_negativo))]
no_comunes_n = [x for x in palabras_negativo if (x not in (palabras_positivo))]

### Se crea el vocabulario final para cada cada categoria

In [33]:
def vocabulario(palabras, head=0):
    serie = pd.Series(palabras)
    frecuencias = serie.value_counts()
    if head > 0:
        return frecuencias.head(head).index  
    return frecuencias.index

In [34]:
vocabulario_positivo = vocabulario(no_comunes_p)
vocabulario_negativo = vocabulario(no_comunes_n)

## Clasificador

In [35]:
def clasificador(texto):
    texto = texto.lower()
    palabras_nuevas = tokenizador_expreg(texto, pattern='\w+') 
    palabras_nuevas = clear_stop_words(palabras_nuevas)
    
    vocabulario_nuevas = vocabulario(palabras_nuevas)
    # print(vocabulario_nuevas)

    cont_p = len([x for x in vocabulario_nuevas if x in vocabulario_positivo])
    cont_n = len([x for x in vocabulario_nuevas if x in vocabulario_negativo])
    
    coincidencias = {"Positivo": cont_p/len(vocabulario_positivo),
            "Negativo": cont_n/len(vocabulario_negativo)}

    if coincidencias["Positivo"] > coincidencias["Negativo"]:
        categoria = "Positivo"
    else:
        categoria = "Negativo"
    
    return coincidencias, categoria

## Evaluación del clasificador

### Se evalúa mediante el calculo del accuracy con todos los datos de testeo

In [23]:
aciertos = 0

for i in df_test.index:
    p, c = clasificador(df_test.loc[i]['Tweet'])

    if c == df_test.loc[i]['Categoria']:
        aciertos += 1
    else:
        print(df_test.loc[i]['Tweet'])
        print('categoria real:', df_test.loc[i]['Categoria'])
        print('categoria pronosticada:', c)
        print('Pertenencia:', p)
        print('-'*40, '\n')

print("accuracy:", aciertos/df_test.shape[0])

@caidodelcatre2 Que cerca che gran gestión

https://t.co/oqVRfFFqq0
categoria real: Positivo
categoria pronosticada: Negativo
Pertenencia: {'Positivo': 0.0, 'Negativo': 0.0008370535714285714}
---------------------------------------- 

🔴#Jujuy | Para nosotros es importante que tanto Alberto Fernández como Cristina Kirchner sean parte de este proceso, aseguró el candidato a Diputado de Unidad para la Victoria, Diego Palmieri.

👉Leelo acá:

https://t.co/AevW4opfWV
categoria real: Positivo
categoria pronosticada: Negativo
Pertenencia: {'Positivo': 0.0, 'Negativo': 0.00027901785714285713}
---------------------------------------- 

@KulfasM @Rurales_FARER @CARBAP_ARG @CRAprensa @ruraltucuman En tanto, desde la Mesa de Enlace de Entre Ríos observan con profunda preocupación el anuncio del presidente Alberto Fernández de reabrir sólo de modo parcial las exportaciones de carne, hecho que significa puntualmente que continúan las restricciones a las mismas y que el…
categoria real: Negativo
categ

## Conclusiones

### El resultado de este clasificador tan simple es sumamente aceptable ya que teniendo un conjunto de 1300 tweets de tan variada redacción y diversos sentidos, se hubiesen esperado resultados mas desfavorables.
### No se tuvo en cuenta en este desarrrollo la presencia de IRONIAS donde el Clasificador mostró una fuerte debilidad. Las palabras que tienen un sentido pero cargado se una intensión inversa a la redactada lleva al error sin ninguna duda a este clasificador.
### Parece que el desvalance en la cantidad de Tweets negativos y positivos parece no tenber mayor influencia en la respuesta del modelo.
### Es importante mencionar que los corpus fueron armados con los Tweets positivos y negativos, dejando sin consideracion a los neutros ya que esta clasificacion fue donde se incluyeron a los Tweets sin contenido de emociones (titulares sin orientación) o los Tweets que no podian ser catalogados como positivos o negatios.